In [1]:
import os
import pickle
from langchain.vectorstores import Pinecone as LangchainPinecone
#|from langchain.vectorstores import Pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from pinecone import Pinecone
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore
from langchain.chat_models import ChatOpenAI  # If using OpenAI, replace if needed
from langchain.llms import HuggingFaceHub  # If using Hugging Face
import warnings
import pandas as pd

/Users/kuldeepchinwan/Projects/LangChainDocumentationChatbot/ai_env/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
warnings.filterwarnings('ignore')

In [3]:
# Load variables from .env file
load_dotenv()

# Load API key from .env file
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")  
PINECONE_ENV = os.getenv("PINECONE_ENV")  
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
index_name = "langchain-docs"

# Initialize Pinecone Client
pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)

# Connect to the Pinecone index
if index_name not in pc.list_indexes().names():
    print(f"Index '{index_name}' not found. Make sure it's created in Pinecone.")
else:
    print(f"Connected to Pinecone index: {index_name}")

index = pc.Index(index_name) 
print(index)

Connected to Pinecone index: langchain-docs


In [4]:
index_stats = index.describe_index_stats()
print(index_stats)

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1113}},
 'total_vector_count': 1113}


In [5]:
# Load the Hugging Face embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Corrected Pinecone Vector Store Initialization
vectorstore = PineconeVectorStore(
    pinecone_api_key = PINECONE_API_KEY, index_name=index_name, embedding=embedding_model
)


print("Pinecone Vector Store is ready!")


Pinecone Vector Store is ready!


In [6]:
#quering from pinecone
query_embedding = embedding_model.embed_query("How does LangChain work?")

vector_results = index.query(
    vector=query_embedding, 
    top_k=5, 
    include_metadata=True
)

print(vector_results)

{'matches': [{'id': '133',
              'metadata': {'source': 'langchain/docs/docs/versions/v0_2/overview.mdx',
                           'text': 'All of which is to say that there’s no '
                                   'large benefits to langchain depending on '
                                   'langchain-community and some obvious '
                                   'downsides: the functionality in langchain '
                                   'should be integration agnostic anyways, '
                                   'langchain-community can’t be properly '
                                   'versioned, and depending on '
                                   'langchain-community increases the '
                                   'vulnerability surface of langchain. For '
                                   'more context about the reason for the '
                                   'organization please see our blog: '
                                   'https://blog.langchai

In [7]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [11]:
#retriever testing wheter it works or not
retrieved_docs = retriever.get_relevant_documents("How does LangChain work?")

print("Retrieved Documents:")
for doc in retrieved_docs:
    print(doc)

Retrieved Documents:
page_content='All of which is to say that there’s no large benefits to langchain depending on langchain-community and some obvious downsides: the functionality in langchain should be integration agnostic anyways, langchain-community can’t be properly versioned, and depending on langchain-community increases the vulnerability surface of langchain. For more context about the reason for the organization please see our blog: https://blog.langchain.dev/langchain-v0-1-0/' metadata={'source': 'langchain/docs/docs/versions/v0_2/overview.mdx'}
page_content='check out our first LangChain Academy course, Introduction to LangGraph, available here. How-to guides Here you’ll find short answers to “How do I….?” types of questions. These how-to guides don’t cover topics in depth – you’ll find that material in the Tutorials and the API Reference. However, these guides will help you quickly accomplish common tasks using chat models, vector stores, and other common LangChain componen

In [12]:
llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-alpha",
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

In [13]:
#question-answering (QA) pipeline using LangChain’s RetrievalQA with a retriever and a language model (LLM)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [14]:
query = "How does LangChain handle document loading?"
response = qa_chain({"query": query})

# Display the response
print("Chatbot Response:")
print(response["result"])

# Show retrieved sources
print("\nRetrieved Documents:")
for doc in response["source_documents"]:
    print(f"Source: {doc.metadata['source']}\n")


Chatbot Response:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

to accepts a query and return documents. In particular, LangChain's retriever class only requires that the _get_relevant_documents method is implemented, which takes a query: str and returns a list of Document objects that are most relevant to the query. The underlying logic used to get relevant documents is specified by the retriever and can be whatever is most useful for the application. A LangChain retriever is a runnable, which is a standard interface is for LangChain components. This means that it has a few common methods, including invoke, that are used to interact with it. A retriever can be invoked with a query: python docs = retriever.invoke(query) Retrievers return a list of Document objects, which have two attributes: page_content: The content of this document. Currently is a string. metadata: Ar

In [15]:
test_queries = [
    "How does LangChain handle document loading?",
    "What are the key components of LangChain?",
    "How do I use LangChain with OpenAI embeddings?",
    "Explain how to fine-tune a chatbot with LangChain.",
    "What is the difference between retriever and generator models?",
    "How can I deploy a LangChain chatbot?"
]


In [16]:
results = []

for query in test_queries:
    response = qa_chain({"query": query})
    
    result_data = {
        "Query": query,
        "Response": response["result"],
        "Sources": [doc.metadata["source"] for doc in response["source_documents"]],
    }
    
    results.append(result_data)

print("Finished Testing Queries!")


HfHubHTTPError: 500 Server Error: Internal Server Error for url: https://router.huggingface.co/hf-inference/models/HuggingFaceH4/zephyr-7b-alpha (Request ID: Root=1-67cd33f0-38be251d7de535ac4d946c8e;734d82c4-7c00-43bd-a92f-dd1e7212f2e9)

Model too busy, unable to get response in less than 60 second(s)

In [38]:
df_results = pd.DataFrame(results)
display(df_results)

,Query,Response,Sources
0,How does LangChain handle document loading?,Use the following pieces of context to answer ...,[]
1,What are the key components of LangChain?,Use the following pieces of context to answer ...,[]
2,How do I use LangChain with OpenAI embeddings?,Use the following pieces of context to answer ...,[]
3,Explain how to fine-tune a chatbot with LangCh...,Use the following pieces of context to answer ...,[]
4,What is the difference between retriever and g...,Use the following pieces of context to answer ...,[]
5,How can I deploy a LangChain chatbot?,Use the following pieces of context to answer ...,[]
